In [8]:
# === KOMÓTKA 1: Wersja v5 - OSTATECZNA - Parser 'Path' skrojony na miarę prawdziwych danych ===

import pandas as pd
import numpy as np
from IPython.display import display

print("--- START: Budowanie słownika na podstawie RZECZYWISTEJ struktury 'Path' ---")

try:
    # --- KROK 1: Wczytanie danych ---
    print("Wczytywanie plików...")
    
    first_row = pd.read_csv('saleflats_2024_dateAdded_polska.csv', sep=',', header=None, on_bad_lines='skip', nrows=1)
    num_columns = first_row.shape[1]
    ulica_id_column_index = num_columns - 1
    
    df_oferty_raw = pd.read_csv(
        'saleflats_2024_dateAdded_polska.csv', sep=',', header=None, on_bad_lines='skip',
        usecols=[ulica_id_column_index], low_memory=False, dtype=str 
    )
    df_oferty_raw.columns = ['UlicaID']
    
    df_lokalizacja_raw = pd.read_csv(
        'lokalizacja.csv', sep=',', header=None, on_bad_lines='skip', low_memory=False,
        names=['LocationID', 'ParentId', 'Name', 'Type', 'Path'], dtype=str,
    )
    print("Pliki wczytane.\n")

    # --- KROK 2: Przygotowanie i czyszczenie ---
    print("Przygotowywanie danych...")
    
    df_oferty_raw['UlicaID'] = pd.to_numeric(df_oferty_raw['UlicaID'], errors='coerce')
    df_oferty_raw.dropna(subset=['UlicaID'], inplace=True)
    unikalne_ulice_id = set(df_oferty_raw['UlicaID'].astype(np.int64).unique())
    
    df_lokalizacja = df_lokalizacja_raw.copy()
    df_lokalizacja['LocationID'] = pd.to_numeric(df_lokalizacja['LocationID'], errors='coerce')
    df_lokalizacja.dropna(subset=['LocationID', 'Path'], inplace=True)
    df_lokalizacja['LocationID'] = df_lokalizacja['LocationID'].astype(np.int64)
    
    df_ulice_do_przetworzenia = df_lokalizacja[df_lokalizacja['LocationID'].isin(unikalne_ulice_id)].copy()
    print(f"Znaleziono {len(df_ulice_do_przetworzenia)} pasujących ulic z ofert w pliku lokalizacji.\n")

    # --- KROK 3: Inteligentne parsowanie ścieżki 'Path' ---
    print("Parsowanie ścieżek wg odkrytej struktury...")

    final_data = []

    for _, row in df_ulice_do_przetworzenia.iterrows():
        path_str = row['Path']
        name_str = row['Name']
        parts = [p.strip() for p in path_str.split(',')]
        
        # Inicjalizacja
        miasto, dzielnica, ulica = None, 'Brak', None

        # 1. Wyciągnij nazwę ulicy
        # Jeśli 'Name' wygląda jak liczba, prawdziwa nazwa jest na końcu 'Path'
        if name_str.isdigit() and len(parts) > 1:
            ulica = parts[-1]
            # Usuwamy ostatni element, żeby nie był brany pod uwagę przy szukaniu miasta/dzielnicy
            parts = parts[:-1]
        else:
            ulica = name_str

        # 2. Logika znajdowania Miasta i Dzielnicy
        if len(parts) >= 2:
            # Sprawdzamy czy drugi element to powiat (kończy się na 'ski' lub 'cki')
            if parts[1].endswith('ski') or parts[1].endswith('cki'):
                if len(parts) >= 3:
                    miasto = parts[2]
                    if len(parts) >= 4:
                        dzielnica = parts[3]
            else: # Drugi element to prawdopodobnie miasto na prawach powiatu (np. Warszawa, Wrocław)
                miasto = parts[1]
                if len(parts) >= 3:
                    dzielnica = parts[2]
        
        # 3. Jeśli udało się znaleźć miasto, dodajemy wpis
        if miasto:
            final_data.append({
                'UlicaID': row['LocationID'],
                'Ulica_Name': ulica,
                'Dzielnica_Name': dzielnica,
                'Miasto_Name': miasto
            })

    slownik_df = pd.DataFrame(final_data)
    print(f"Udało się zbudować słownik dla {len(slownik_df)} ulic na podstawie kolumny 'Path'.\n")

    # --- KROK 4: Zapis i wyświetlenie ---
    if not slownik_df.empty:
        # Usuwamy ewentualne duplikaty i sortujemy dla porządku
        slownik_df.drop_duplicates(subset=['UlicaID'], inplace=True)
        slownik_df.sort_values(by=['Miasto_Name', 'Dzielnica_Name', 'Ulica_Name'], inplace=True)
        
        output_filename = 'slownik_finalny_z_hierarchia_polska.csv'
        print(f"Zapisywanie słownika do pliku: {output_filename}...")
        slownik_df.to_csv(output_filename, index=False, sep=';')
        print("Zapis zakończony sukcesem!")

        print("\nPróbka 30 losowych wierszy z finalnego słownika:")
        display(slownik_df.sample(min(30, len(slownik_df))))
    else:
        print("\nNiestety, nie udało się zbudować słownika. Problem z danymi jest zbyt poważny.")

except Exception as e:
    print(f"Wystąpił nieoczekiwany błąd: {e}")
    import traceback
    traceback.print_exc()

--- START: Budowanie słownika na podstawie RZECZYWISTEJ struktury 'Path' ---
Wczytywanie plików...
Pliki wczytane.

Przygotowywanie danych...
Znaleziono 1999 pasujących ulic z ofert w pliku lokalizacji.

Parsowanie ścieżek wg odkrytej struktury...
Udało się zbudować słownik dla 1896 ulic na podstawie kolumny 'Path'.

Zapisywanie słownika do pliku: slownik_finalny_z_hierarchia_polska.csv...
Zapis zakończony sukcesem!

Próbka 30 losowych wierszy z finalnego słownika:


,UlicaID,Ulica_Name,Dzielnica_Name,Miasto_Name
1,76,Kłodzki,Brak,Kłodzki
1002,31961,Sikórz,Sikórz,Brudzeń duży
134,2003,Krzyż wielkopolski,Brak,Krzyż wielkopolski
659,16319,Suszka,Suszka,Biszcza
64,511,Adamów,Brak,Adamów
562,13680,Wielgołęka,Wielgołęka,Gołymin-ośrodek
650,16129,Gałkówek-parcela,Koluszki,Łódzki wschodni
1147,52845,Popowo,Popowo,Stegna
441,10639,Kamień,Kamień,Kamień
339,7895,Bzowo,Bzowo,Kobylnica
